In [305]:
# Import BeautifulSoup to parse the html and urlopen to req html from a site
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
import time
import re
import pandas as pd
import numpy as np

In [243]:
# Setup the parameters for url, chromedriver path and how long it takes your device to render the updated restaurants
postcode = 'EC1R0HX'
url = f'https://deliveroo.co.uk/restaurants/london/clerkenwell?postcode={postcode}&collection=all-restaurants'
chrome_driver_path = r'/home/selvino/chrome-driver/chromedriver'
initial_load_time = 2
scroll_pause_time = 1.5

In [244]:
# Assign the webdriver to a variable which simulates an instance of Chrome on the url of choice
# The 2 seconds sleep gives the webpage a 2 second break to load (can be changed)
# Get the height of the devices screen to know how much needs to be scrolled each time
# Tell the browser to click on the accept cookies button and the £10 off button

driver = webdriver.Chrome(executable_path=chrome_driver_path)
driver.get(url)
time.sleep(initial_load_time)
screen_height = driver.execute_script('return window.screen.height;')
driver.execute_script('document.querySelector(".accept-cookies-button").click()')
driver.execute_script('document.querySelectorAll(".ccl-d0484b0360a2b432")[1].click()')

In [245]:
# Create a whie loop that scrolls through the website. This is done so the full infinite scrolling webpage renders and is in the html source

i = 1
while True:
    driver.execute_script("window.scrollTo(0, {screen_height} * {i});".format(screen_height=screen_height, i = i))
    i += 1
    time.sleep(scroll_pause_time)
    scroll_height = driver.execute_script("return document.body.scrollHeight")
    if screen_height * i > scroll_height: 
        break

In [246]:
# Create a beautiful soup object that parses the html and has useful methods for reading the text
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [346]:
# Need to loop through the text on the source web page and make a list of all the restaurants' data
# The identifier variables are used to know where in the webpage to start looking and what seperates restaurants

start_identifier = 'restaurants'
restaurant_identifier= 'miles away'
restaurant_identifier_2 = 'mile away'

start_point = soup.get_text().find(start_identifier) + len(start_identifier)
useful_text = soup.get_text()[start_point:]
restaurants = []
search = True
search_start = 0
while search:
    end_index = useful_text.find(restaurant_identifier, search_start)
    end_index2 = useful_text.find(restaurant_identifier_2, search_start)
    if (end_index == -1) & (end_index2 == -1):
        break
    else:
        if (end_index > end_index2) & (end_index2 != -1):
            end_index = end_index2
        
        search_end = end_index + len(restaurant_identifier)
        restaurant_data = useful_text[search_start : search_end]
        restaurant_dict = {}
        
        #Extract Cuisines and Distance
        cuisines_raw = restaurant_data.split('·')
        cuisine_list = [x for x in cuisines_raw if len(x) > 1]
        if len(cuisine_list) > 1: 
            raw_last_cuisine = cuisine_list[-1]
            last_cuisine = raw_last_cuisine[:index_first_number(raw_last_cuisine)]
            distance = raw_last_cuisine[index_first_number(raw_last_cuisine):]
            cuisine_list = cuisine_list[1:-1]
            cuisine_list.append(last_cuisine)
            restaurant_dict['Cuisines'] = cuisine_list
            restaurant_dict['Distance'] = distance
        
        #Extract Ratings
        if len(cuisines_raw) > 1:
            raw_rating = cuisines_raw[0]
            seperated_rating = raw_rating.split('.')
            if len(seperated_rating) > 1:
                rating = seperated_rating[0][-1] + '.' + seperated_rating[1]
                restaurant_dict['Rating'] = rating
        
        #Extract Name and Delivery Time
        raw_name = seperated_rating[0][:-1]
        restaurant_dict['Name'] = raw_name.split('–')[-1][6:]
        restaurant_dict['Delivery Time'] = seperated_rating[0].split(r'\n')
        
            
        if restaurant_dict:
            restaurants.append(restaurant_dict)
        search_start = search_end + 1
        

In [349]:
restaurants

[{'Cuisines': ['American', 'Burgers', 'Milkshakes'],
  'Distance': '0.7 miles away',
  'Rating': '4.7 Excellent(500+)',
  'Name': 'Five Guys - Burger and Fries',
  'Delivery Time': ['Free\ndelivery15 – 25minFive Guys - Burger and Fries4']},
 {'Cuisines': ['Mexican', 'Burritos', 'Salads', 'Chicken'],
  'Distance': '0.7 miles away',
  'Rating': '4.6 Excellent(500+)',
  'Name': 'Chipotle Mexican Grill',
  'Delivery Time': ['Free deliveryFree\ndelivery10 – 20minChipotle Mexican Grill4']},
 {'Cuisines': ['Salads', 'Poke', 'Hawaiian', 'Seafood'],
  'Distance': '0.7 miles away',
  'Rating': '4.8 Excellent(500+)',
  'Name': '🏄Honi Poke - Angel🏄',
  'Delivery Time': ['Free deliveryFree\ndelivery10 – 20min🏄Honi Poke - Angel🏄4']},
 {'Name': '🏄Honi Poke - Angel🏄',
  'Delivery Time': ['Free deliveryFree\ndelivery10 – 20min🏄Honi Poke - Angel🏄4']},
 {'Cuisines': ['German', 'Kebab'],
  'Distance': '0.4 miles away',
  'Rating': '4.6 Excellent(500+)',
  'Name': 'German Doner Kebab',
  'Delivery Time': [

In [307]:
def index_first_number(string):
    while True: 
        for i in range(0, len(string)):
            if string[i].isdigit():
                return string.find(string[i])
        return 'None'

In [344]:
a = r'\beans'
a[0:]

'\\beans'